## Introduction

The objective of this project is to address the issue of inequality in educational opportunities among U.S. high schools. Our main focus is on the average performance of students on standardized tests such as the ACT or SAT, which are integral to the college application process. Through our research, we seek to determine whether socioeconomic factors influence school performance on these exams.

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno
import csv

# Train-test splits
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV

# Model preprocessing
from sklearn.preprocessing import StandardScaler

# Modeling
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Model preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

# Model metrics and analysis
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

# Models
from sklearn import linear_model

## The data

This project utilizes two data sets. The primary data set is the EdGap data set from [EdGap.org](https://www.edgap.org/#5/37.875/-96.987). This data set from 2016 includes information about average ACT or SAT scores for schools and several socioeconomic characteristics of the school district. The secondary data set is basic information about each school from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp).





### EdGap data

All socioeconomic data (household income, unemployment, adult educational attainment, and family structure) are from the Census Bureau's American Community Survey. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) report that ACT and SAT score data is from each state's department of education or some other public data release. The nature of the other public data release is not known.

The quality of the census data and the department of education data can be assumed to be reasonably high. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) do not indicate that they processed the data in any way. The data were assembled by the [EdGap.org](https://www.edgap.org/#5/37.875/-96.987) team, so there is always the possibility for human error. Given the public nature of the data, we would be able to consult the original data sources to check the quality of the data if we had any questions.

### School information data

The school information data is from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp). This data set consists of basic identifying information about schools and can be assumed to be of reasonably high quality. As for the EdGap.org data, the school information data is public, so we would be able to consult the original data sources to check the quality of the data if we had any questions.


## Load the data

Load the EdGap
 data set

In [ ]:
edgap = pd.read_excel('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/education/EdGap_data.xlsx')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Load the school information data

In [ ]:
!wget https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0

--2023-05-07 18:18:33--  https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv [following]
--2023-05-07 18:18:34--  https://www.dropbox.com/s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf8967908cf4fecde7c4962a0a9.dl.dropboxusercontent.com/cd/0/inline/B7kKbfRjOTqtScawwOdL7ctUrZZ8OJ2beeR_bTc03hhZn1E9P1MggillvPQFOpzPhpuwKNlrmLBrIPQhLcNJNp-SQyhhvt2FVmSOizyCNuUcPsbwUw7etyGbNa6i0-PzMhkgqZLxlaoRlnfPo-cObd33P2ZAkl9BgA5elNHFhWYYKA/file# [following]
--2023-05-07 18:18:34--  https://ucf8967908cf4fecde7c4962a0a9.dl.dropboxusercontent.com/cd/0/inline/B7kKbfRjO

In [ ]:
school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding= 'unicode_escape')

<ipython-input-57-54e0747c4ccf>:1: DtypeWarning: Columns (6,9,14,15,18,19,21,22,25,26,29,31,35,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding= 'unicode_escape')


## Inspect the contents of each data set.

In this section we are inspects the contents of each data set, starting off by looking at the head of each data frame.

In [ ]:
edgap.head()

,NCESSCH School ID,CT Unemployment Rate,CT Pct Adults with College Degree,CT Pct Childre In Married Couple Family,CT Median Household Income,School ACT average (or equivalent if SAT score),School Pct Free and Reduced Lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [ ]:
school_info.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,...,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2016-2017,1,ALABAMA,AL,Sequoyah Sch - Chalkville Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
1,2016-2017,1,ALABAMA,AL,Camps,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
2,2016-2017,1,ALABAMA,AL,Det Ctr,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
3,2016-2017,1,ALABAMA,AL,Wallace Sch - Mt Meigs Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
4,2016-2017,1,ALABAMA,AL,McNeel Sch - Vacca Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported


We are using the `info` method to check the data types, size of the data frame, and numbers of missing values.

In [ ]:
edgap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7986 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   NCESSCH School ID                                7986 non-null   int64  
 1   CT Unemployment Rate                             7972 non-null   float64
 2   CT Pct Adults with College Degree                7973 non-null   float64
 3   CT Pct Childre In Married Couple Family          7961 non-null   float64
 4   CT Median Household Income                       7966 non-null   float64
 5   School ACT average (or equivalent if SAT score)  7986 non-null   float64
 6   School Pct Free and Reduced Lunch                7986 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 436.9 KB


It appears that there is some missing points in the school_info()

In [ ]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102183 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102183 non-null  object 
 1   FIPST                102183 non-null  int64  
 2   STATENAME            102183 non-null  object 
 3   ST                   102183 non-null  object 
 4   SCH_NAME             102183 non-null  object 
 5   LEA_NAME             102183 non-null  object 
 6   STATE_AGENCY_NO      102183 non-null  object 
 7   UNION                2533 non-null    float64
 8   ST_LEAID             102183 non-null  object 
 9   LEAID                102183 non-null  object 
 10  ST_SCHID             102183 non-null  object 
 11  NCESSCH              102181 non-null  float64
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3         

It appears that there is some missing points in the school_info()

## Convert data types, if necessary


We need to join two DataFrames using the school identity as the key, which is represented by the NCESSCH column. However, the column has different names in the two DataFrames and is stored as an int64 in one DataFrame and a float64 in the other.

To merge the two DataFrames, we'll first cast the NCESSCH column in the DataFrame with the float64 datatype to an int64. Before casting, we must drop any rows where the NCESSCH value is NaN.


In [ ]:
school_info = school_info[school_info['NCESSCH'].isna() == False]

In [ ]:
school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')

<ipython-input-63-006ea9997674>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')


In [ ]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102181 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102181 non-null  object 
 1   FIPST                102181 non-null  int64  
 2   STATENAME            102181 non-null  object 
 3   ST                   102181 non-null  object 
 4   SCH_NAME             102181 non-null  object 
 5   LEA_NAME             102181 non-null  object 
 6   STATE_AGENCY_NO      102181 non-null  object 
 7   UNION                2531 non-null    float64
 8   ST_LEAID             102181 non-null  object 
 9   LEAID                102181 non-null  object 
 10  ST_SCHID             102181 non-null  object 
 11  NCESSCH              102181 non-null  int64  
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3         

## Select relevant subsets of the data

Since the school information data set contains a lot of information, we only need the year school identity, location, and school type information.

In [ ]:
school_info = school_info[['SCHOOL_YEAR', 'NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL']]

In [ ]:
school_info.head()

,SCHOOL_YEAR,NCESSCH,MSTATE,MZIP,SCH_TYPE_TEXT,LEVEL
0,2016-2017,10000200277,AL,35220,Alternative School,High
1,2016-2017,10000201667,AL,36057,Alternative School,High
2,2016-2017,10000201670,AL,36057,Alternative School,High
3,2016-2017,10000201705,AL,36057,Alternative School,High
4,2016-2017,10000201706,AL,35206,Alternative School,High


## Rename columns

Renaming the columns to follow best practices of being lowercase, snake_case

In [ ]:
edgap = edgap.rename(columns={"NCESSCH School ID":"id", 
              "CT Pct Adults with College Degree":"percent_college",        
              "CT Unemployment Rate":"rate_unemployment", 
              "CT Pct Childre In Married Couple Family":"percent_married",
              "CT Median Household Income":"median_income",
              "School ACT average (or equivalent if SAT score)":"average_act",
              "School Pct Free and Reduced Lunch":"percent_lunch"})

In [ ]:
school_info = school_info.rename(columns={'SCHOOL_YEAR':'year', 
                                          'NCESSCH':'id', 
                                          'MSTATE':'state',
                                          'MZIP':'zip_code',
                                          'SCH_TYPE_TEXT':'school_type',
                                          'LEVEL':'school_level'})

In [ ]:
edgap.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [ ]:
school_info.head()

,year,id,state,zip_code,school_type,school_level
0,2016-2017,10000200277,AL,35220,Alternative School,High
1,2016-2017,10000201667,AL,36057,Alternative School,High
2,2016-2017,10000201670,AL,36057,Alternative School,High
3,2016-2017,10000201705,AL,36057,Alternative School,High
4,2016-2017,10000201706,AL,35206,Alternative School,High


## Join data frames 

Here we are joing the two data sets with a left join 

In [ ]:
edgap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7986 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7986 non-null   int64  
 1   rate_unemployment  7972 non-null   float64
 2   percent_college    7973 non-null   float64
 3   percent_married    7961 non-null   float64
 4   median_income      7966 non-null   float64
 5   average_act        7986 non-null   float64
 6   percent_lunch      7986 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 436.9 KB


In [ ]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102181 entries, 0 to 102182
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   year          102181 non-null  object
 1   id            102181 non-null  int64 
 2   state         102181 non-null  object
 3   zip_code      102181 non-null  object
 4   school_type   102179 non-null  object
 5   school_level  102179 non-null  object
dtypes: int64(1), object(5)
memory usage: 5.5+ MB


In [ ]:
df = edgap.merge(school_info, how = 'left', on='id')

In [ ]:
df.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,2016-2017,DE,19804,Regular School,High
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,2016-2017,DE,19709,Regular School,High
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,2016-2017,DE,19709,Regular School,High
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,2016-2017,DE,19958,Regular School,High
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,2016-2017,DE,19934,Regular School,High


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7986 entries, 0 to 7985
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7986 non-null   int64  
 1   rate_unemployment  7972 non-null   float64
 2   percent_college    7973 non-null   float64
 3   percent_married    7961 non-null   float64
 4   median_income      7966 non-null   float64
 5   average_act        7986 non-null   float64
 6   percent_lunch      7986 non-null   float64
 7   year               7898 non-null   object 
 8   state              7898 non-null   object 
 9   zip_code           7898 non-null   object 
 10  school_type        7898 non-null   object 
 11  school_level       7898 non-null   object 
dtypes: float64(6), int64(1), object(5)
memory usage: 811.1+ KB


##Additonal Step - HouseHold number

In [ ]:
dh = pd.read_csv('2017HouseData.csv', skiprows=1)

The code above removes the first row, so we can get the specific names and get rid of ones not needed 

In [ ]:
dh.head()

,Geography,Geographic Area Name,Estimate!!Occupied housing units!!Occupied housing units,Annotation of Estimate!!Occupied housing units!!Occupied housing units,Margin of Error!!Occupied housing units MOE!!Occupied housing units,Annotation of Margin of Error!!Occupied housing units MOE!!Occupied housing units,Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD SIZE!!1-person household,Annotation of Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD SIZE!!1-person household,Margin of Error!!Occupied housing units MOE!!Occupied housing units!!HOUSEHOLD SIZE!!1-person household,Annotation of Margin of Error!!Occupied housing units MOE!!Occupied housing units!!HOUSEHOLD SIZE!!1-person household,...,Annotation of Margin of Error!!Percent renter-occupied housing units MOE!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!With related children of householder under 18 years!!With own children of householder under 18 years!!6 to 17 years only,Estimate!!Percent renter-occupied housing units!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!With related children of householder under 18 years!!No own children of householder under 18 years,Annotation of Estimate!!Percent renter-occupied housing units!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!With related children of householder under 18 years!!No own children of householder under 18 years,Margin of Error!!Percent renter-occupied housing units MOE!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!With related children of householder under 18 years!!No own children of householder under 18 years,Annotation of Margin of Error!!Percent renter-occupied housing units MOE!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!With related children of householder under 18 years!!No own children of householder under 18 years,Estimate!!Percent renter-occupied housing units!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!No related children of householder under 18 years,Annotation of Estimate!!Percent renter-occupied housing units!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!No related children of householder under 18 years,Margin of Error!!Percent renter-occupied housing units MOE!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!No related children of householder under 18 years,Annotation of Margin of Error!!Percent renter-occupied housing units MOE!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!No related children of householder under 18 years,Unnamed: 914
0,8600000US00601,ZCTA5 00601,5818,NaN,249,NaN,1416,NaN,189,NaN,...,NaN,1.0,NaN,0.9,NaN,58.0,NaN,5.0,NaN,NaN
1,8600000US00602,ZCTA5 00602,12719,NaN,374,NaN,2938,NaN,357,NaN,...,NaN,4.1,NaN,2.7,NaN,58.9,NaN,6.4,NaN,NaN
2,8600000US00603,ZCTA5 00603,19009,NaN,503,NaN,5037,NaN,398,NaN,...,NaN,2.8,NaN,1.2,NaN,56.8,NaN,3.9,NaN,NaN
3,8600000US00606,ZCTA5 00606,1959,NaN,154,NaN,425,NaN,103,NaN,...,NaN,0.0,NaN,8.1,NaN,52.6,NaN,12.4,NaN,NaN
4,8600000US00610,ZCTA5 00610,9120,NaN,375,NaN,2208,NaN,308,NaN,...,NaN,5.2,NaN,3.6,NaN,62.7,NaN,7.5,NaN,NaN


In [ ]:
dh.head()

,Geography,Geographic Area Name,Estimate!!Occupied housing units!!Occupied housing units,Annotation of Estimate!!Occupied housing units!!Occupied housing units,Margin of Error!!Occupied housing units MOE!!Occupied housing units,Annotation of Margin of Error!!Occupied housing units MOE!!Occupied housing units,Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD SIZE!!1-person household,Annotation of Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD SIZE!!1-person household,Margin of Error!!Occupied housing units MOE!!Occupied housing units!!HOUSEHOLD SIZE!!1-person household,Annotation of Margin of Error!!Occupied housing units MOE!!Occupied housing units!!HOUSEHOLD SIZE!!1-person household,...,Annotation of Margin of Error!!Percent renter-occupied housing units MOE!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!With related children of householder under 18 years!!With own children of householder under 18 years!!6 to 17 years only,Estimate!!Percent renter-occupied housing units!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!With related children of householder under 18 years!!No own children of householder under 18 years,Annotation of Estimate!!Percent renter-occupied housing units!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!With related children of householder under 18 years!!No own children of householder under 18 years,Margin of Error!!Percent renter-occupied housing units MOE!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!With related children of householder under 18 years!!No own children of householder under 18 years,Annotation of Margin of Error!!Percent renter-occupied housing units MOE!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!With related children of householder under 18 years!!No own children of householder under 18 years,Estimate!!Percent renter-occupied housing units!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!No related children of householder under 18 years,Annotation of Estimate!!Percent renter-occupied housing units!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!No related children of householder under 18 years,Margin of Error!!Percent renter-occupied housing units MOE!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!No related children of householder under 18 years,Annotation of Margin of Error!!Percent renter-occupied housing units MOE!!Occupied housing units!!FAMILY TYPE AND PRESENCE OF OWN CHILDREN!!No related children of householder under 18 years,Unnamed: 914
0,8600000US00601,ZCTA5 00601,5818,NaN,249,NaN,1416,NaN,189,NaN,...,NaN,1.0,NaN,0.9,NaN,58.0,NaN,5.0,NaN,NaN
1,8600000US00602,ZCTA5 00602,12719,NaN,374,NaN,2938,NaN,357,NaN,...,NaN,4.1,NaN,2.7,NaN,58.9,NaN,6.4,NaN,NaN
2,8600000US00603,ZCTA5 00603,19009,NaN,503,NaN,5037,NaN,398,NaN,...,NaN,2.8,NaN,1.2,NaN,56.8,NaN,3.9,NaN,NaN
3,8600000US00606,ZCTA5 00606,1959,NaN,154,NaN,425,NaN,103,NaN,...,NaN,0.0,NaN,8.1,NaN,52.6,NaN,12.4,NaN,NaN
4,8600000US00610,ZCTA5 00610,9120,NaN,375,NaN,2208,NaN,308,NaN,...,NaN,5.2,NaN,3.6,NaN,62.7,NaN,7.5,NaN,NaN


Upon reviewing all of the rows and columns, we came to the realization that certain ones were unnecessary. As a result, we proceeded to remove those superfluous columns from the Excel sheet.

Additionally, we renamed all of the columns in the Excel sheet. The result is down below

In [ ]:
dh = pd.read_csv('EditedData.csv' , skiprows = 1)

In [ ]:
dh.head()

,Geography,Zip code,Total estimate of occupied housing unit,One person,Two people,Three people,Four People
0,8600000US00601,ZCTA5 00601,5818,1416,1909,1318,1175
1,8600000US00602,ZCTA5 00602,12719,2938,4219,2436,3126
2,8600000US00603,ZCTA5 00603,19009,5037,6152,3999,3821
3,8600000US00606,ZCTA5 00606,1959,425,701,337,496
4,8600000US00610,ZCTA5 00610,9120,2208,2855,1793,2264


Since every zip code starts with ZCTA5 we want to remove that

In [ ]:
dh['Zip code'] = dh['Zip code'].str.split('ZCTA5 ').str[1]


In [ ]:
dh["Zip code"] = dh["Zip code"].astype(str)

In [ ]:
dh.head()

,Geography,Zip code,Total estimate of occupied housing unit,One person,Two people,Three people,Four People
0,8600000US00601,00601,5818,1416,1909,1318,1175
1,8600000US00602,00602,12719,2938,4219,2436,3126
2,8600000US00603,00603,19009,5037,6152,3999,3821
3,8600000US00606,00606,1959,425,701,337,496
4,8600000US00610,00610,9120,2208,2855,1793,2264


##### $\rightarrow$ Renaming Data2017 

In [ ]:
dh = dh.rename(columns={'Geography':'geography', 
                                          'Zip code':'zip_code', 
                                          'Total estimate of occupied housing unit':'total_estimate',
                                          'One person':'one_person',
                                          'Two people':'two_people',
                                           'Three people':'three_people',
                                          'Four People':'four_people'})

In [ ]:
dh.head()

,geography,zip_code,total_estimate,one_person,two_people,three_people,four_people
0,8600000US00601,00601,5818,1416,1909,1318,1175
1,8600000US00602,00602,12719,2938,4219,2436,3126
2,8600000US00603,00603,19009,5037,6152,3999,3821
3,8600000US00606,00606,1959,425,701,337,496
4,8600000US00610,00610,9120,2208,2855,1793,2264


##Identifying and Eliminating NaN/Out of bounds Values

Here we are filtering, removing and eliminating specifc values.

Filtering to only have "school_level == High"

In [ ]:
df = df.loc[df['school_level'] == 'High']

Filtering the average act to be greater than or equal to 1 since anything less is not needed. 

In [ ]:
df = df.loc[df['average_act' ] >= 1]

Similarly we are filtering percent lunch to only include values greater than zero 

In [ ]:
df.loc[df['percent_lunch'] < 0, 'percent_lunch'] = np.nan 

In [ ]:
df.isna().sum()

id                    0
rate_unemployment    12
percent_college      11
percent_married      20
median_income        16
average_act           0
percent_lunch        20
year                  0
state                 0
zip_code              0
school_type           0
school_level          0
dtype: int64

In [ ]:
df.isna().mean().round(4)*100

id                   0.00
rate_unemployment    0.17
percent_college      0.15
percent_married      0.28
median_income        0.22
average_act          0.00
percent_lunch        0.28
year                 0.00
state                0.00
zip_code             0.00
school_type          0.00
school_level         0.00
dtype: float64

##Merging the orginal & added subset

Doing a left join with the orginal and added subset to get all records from the left side that match rows from the right tables zip codes

In [ ]:
dh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 0 to 33119
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   geography       33120 non-null  object
 1   zip_code        33120 non-null  object
 2   total_estimate  33120 non-null  int64 
 3   one_person      33120 non-null  int64 
 4   two_people      33120 non-null  int64 
 5   three_people    33120 non-null  int64 
 6   four_people     33120 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 1.8+ MB


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7227 entries, 0 to 7985
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7227 non-null   int64  
 1   rate_unemployment  7215 non-null   float64
 2   percent_college    7216 non-null   float64
 3   percent_married    7207 non-null   float64
 4   median_income      7211 non-null   float64
 5   average_act        7227 non-null   float64
 6   percent_lunch      7207 non-null   float64
 7   year               7227 non-null   object 
 8   state              7227 non-null   object 
 9   zip_code           7227 non-null   object 
 10  school_type        7227 non-null   object 
 11  school_level       7227 non-null   object 
dtypes: float64(6), int64(1), object(5)
memory usage: 734.0+ KB


In [ ]:
df["zip_code"] = df["zip_code"].astype(str)

In [ ]:
df = df.merge(dh, how = 'left', on='zip_code')

In [ ]:
df.head(25)

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level,geography,total_estimate,one_person,two_people,three_people,four_people
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,2016-2017,DE,19804,Regular School,High,8600000US19804,6901.0,2441.0,2133.0,990.0,1337.0
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,2016-2017,DE,19709,Regular School,High,8600000US19709,12810.0,2155.0,4377.0,2204.0,4074.0
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,2016-2017,DE,19709,Regular School,High,8600000US19709,12810.0,2155.0,4377.0,2204.0,4074.0
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,2016-2017,DE,19958,Regular School,High,8600000US19958,10894.0,3140.0,5401.0,1114.0,1239.0
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,2016-2017,DE,19934,Regular School,High,8600000US19934,5053.0,1040.0,1990.0,762.0,1261.0
5,100019000050,0.080120,0.673492,0.483333,50649.0,17.034188,0.425118,2016-2017,DE,19904,Regular School,High,8600000US19904,12649.0,3419.0,3957.0,1984.0,3289.0
6,100020000238,0.075058,0.750751,NaN,12825.0,18.387057,0.338928,2016-2017,DE,19711,Regular School,High,8600000US19711,19087.0,4744.0,7443.0,3124.0,3776.0
7,100020000239,0.075424,0.616605,0.595445,59861.0,15.544567,0.372116,2016-2017,DE,19702,Regular School,High,8600000US19702,19350.0,4412.0,6375.0,3919.0,4644.0
8,100020000240,0.114201,0.594220,0.672059,62078.0,14.980464,0.352801,2016-2017,DE,19713,Regular School,High,8600000US19713,11965.0,4109.0,3684.0,1883.0,2289.0
9,100023000209,0.039931,0.540456,0.840810,61031.0,15.105006,0.358939,2016-2017,DE,19720,Regular School,High,8600000US19720,21362.0,5716.0,7134.0,3505.0,5007.0


We want to remove the geography column from the dataframe since it doesnt provide any details we need. It just has the zip code with "8600000US. In that case we are going to use the drop() method to remove it. The axis=1 parameter specifies that we want to remove a column (as opposed to a row, which would be axis=0).

In [ ]:
df.drop("geography", axis=1, inplace=True)

In [ ]:
df.head(15)

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level,total_estimate,one_person,two_people,three_people,four_people
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,2016-2017,DE,19804,Regular School,High,6901.0,2441.0,2133.0,990.0,1337.0
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,2016-2017,DE,19709,Regular School,High,12810.0,2155.0,4377.0,2204.0,4074.0
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,2016-2017,DE,19709,Regular School,High,12810.0,2155.0,4377.0,2204.0,4074.0
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,2016-2017,DE,19958,Regular School,High,10894.0,3140.0,5401.0,1114.0,1239.0
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,2016-2017,DE,19934,Regular School,High,5053.0,1040.0,1990.0,762.0,1261.0
5,100019000050,0.080120,0.673492,0.483333,50649.0,17.034188,0.425118,2016-2017,DE,19904,Regular School,High,12649.0,3419.0,3957.0,1984.0,3289.0
6,100020000238,0.075058,0.750751,NaN,12825.0,18.387057,0.338928,2016-2017,DE,19711,Regular School,High,19087.0,4744.0,7443.0,3124.0,3776.0
7,100020000239,0.075424,0.616605,0.595445,59861.0,15.544567,0.372116,2016-2017,DE,19702,Regular School,High,19350.0,4412.0,6375.0,3919.0,4644.0
8,100020000240,0.114201,0.594220,0.672059,62078.0,14.980464,0.352801,2016-2017,DE,19713,Regular School,High,11965.0,4109.0,3684.0,1883.0,2289.0
9,100023000209,0.039931,0.540456,0.840810,61031.0,15.105006,0.358939,2016-2017,DE,19720,Regular School,High,21362.0,5716.0,7134.0,3505.0,5007.0


##Train Test split

##### $\rightarrow$ In the matrix predictor variable `x` is all columns except `id` and `average_act` 

#### Get input and output variables.

In [ ]:
x = df[df.columns.difference(['id','average_act', 'total_estimate', 'school_level', 'school_type'])]

In [ ]:
x.head()

,four_people,median_income,one_person,percent_college,percent_lunch,percent_married,rate_unemployment,state,three_people,two_people,year,zip_code
0,1337.0,42820.0,2441.0,0.445283,0.066901,0.346495,0.117962,DE,990.0,2133.0,2016-2017,19804
1,4074.0,89320.0,2155.0,0.662765,0.112412,0.767619,0.063984,DE,2204.0,4377.0,2016-2017,19709
2,4074.0,84140.0,2155.0,0.701864,0.096816,0.713090,0.056460,DE,2204.0,4377.0,2016-2017,19709
3,1239.0,56500.0,3140.0,0.692062,0.296960,0.641283,0.044739,DE,1114.0,5401.0,2016-2017,19958
4,1261.0,54015.0,1040.0,0.640060,0.262641,0.834402,0.077014,DE,762.0,1990.0,2016-2017,19934


##### $\rightarrow$ Defining the output variable `y` to be `average_act`.

In [ ]:
y = df[['average_act']]

In [ ]:
y.head()

,average_act
0,20.433455
1,19.498168
2,19.554335
3,17.737485
4,18.245421


#### Train and test splits

##### $\rightarrow$ Here we are splitting the data into training and testing sets. Keeping 20% of the data for the test set & rest for the train

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,  test_size=.2, random_state = 1)

In [ ]:
print(x_train.shape, x_test.shape)

(5781, 12) (1446, 12)


#### Imputation

Defining an imputer to use later on called "imputer"

In [ ]:
imputer = IterativeImputer()

In [ ]:
imputer.fit(x_train.loc[:,['four_people','two_people', 'one_person', 'three_people', 'median_income', 'percent_college',	'percent_lunch',	'percent_married',	'rate_unemployment']])

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


IterativeImputer()

Imputing the missing values in the training data.

In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5781 entries, 3283 to 5157
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   four_people        5252 non-null   float64
 1   median_income      5766 non-null   float64
 2   one_person         5252 non-null   float64
 3   percent_college    5770 non-null   float64
 4   percent_lunch      5764 non-null   float64
 5   percent_married    5763 non-null   float64
 6   rate_unemployment  5769 non-null   float64
 7   state              5781 non-null   object 
 8   three_people       5252 non-null   float64
 9   two_people         5252 non-null   float64
 10  year               5781 non-null   object 
 11  zip_code           5781 non-null   object 
dtypes: float64(9), object(3)
memory usage: 587.1+ KB


The line of code below is filling missing values in the 'median_income' to 'rate_unemployment' columns of the pandas dataframe x_train using an imputer object. 

In [ ]:
x_train.loc[:, ['four_people','two_people', 'one_person', 'three_people', 'median_income', 'percent_college',	'percent_lunch',	'percent_married',	'rate_unemployment']] = imputer.transform(x_train.loc[:,['four_people','two_people','one_person', 'three_people', 'median_income', 'percent_college',	'percent_lunch',	'percent_married',	'rate_unemployment']])

Now missing values in train should be zero

In [ ]:
x_train.isna().sum()

four_people          0
median_income        0
one_person           0
percent_college      0
percent_lunch        0
percent_married      0
rate_unemployment    0
state                0
three_people         0
two_people           0
year                 0
zip_code             0
dtype: int64

Now doing something similar for the Test data set. First checking missing values, filling and rechecking

In [ ]:
x_test.isna().sum()

four_people          121
median_income          1
one_person           121
percent_college        0
percent_lunch          3
percent_married        2
rate_unemployment      0
state                  0
three_people         121
two_people           121
year                   0
zip_code               0
dtype: int64

In [ ]:
x_test.loc[:, ['four_people','two_people', 'one_person', 'three_people', 'median_income', 'percent_college',	'percent_lunch',	'percent_married',	'rate_unemployment']] = imputer.transform(x_test.loc[:,['four_people','two_people','one_person', 'three_people', 'median_income', 'percent_college',	'percent_lunch',	'percent_married',	'rate_unemployment']])

In [ ]:
x_test.isna().sum()

four_people          0
median_income        0
one_person           0
percent_college      0
percent_lunch        0
percent_married      0
rate_unemployment    0
state                0
three_people         0
two_people           0
year                 0
zip_code             0
dtype: int64

##Joining (X & Y) Train & Test

This line of code is joining two pandas dataframes x_train and y_train horizontally based on their index and creating a new dataframe df_train.

In [ ]:
df_train = x_train.join(y_train)

In [ ]:
df_train.head()

,four_people,median_income,one_person,percent_college,percent_lunch,percent_married,rate_unemployment,state,three_people,two_people,year,zip_code,average_act
3283,1962.794089,41793.0,2554.41932,0.602419,0.542056,0.574034,0.111111,NJ,1400.381764,2852.091229,2016-2017,7306,16.538462
1576,474.000000,38173.0,699.00000,0.469225,0.339655,0.711429,0.135246,IN,417.000000,866.000000,2016-2017,47567,20.367521
5261,422.000000,39635.0,1021.00000,0.567361,0.270175,0.694514,0.083419,PA,441.000000,991.000000,2016-2017,15853,20.347985
2928,286.000000,40978.0,276.00000,0.467614,0.315556,0.766901,0.062531,MO,178.000000,467.000000,2016-2017,64644,21.600000
349,863.000000,36875.0,1247.00000,0.604470,0.548410,0.803435,0.071429,FL,704.000000,2055.000000,2016-2017,34669,21.056166


Similar to what we did for df_train we are doing it to df_test

In [ ]:
df_test = x_test.join(y_test)

In [ ]:
df_test.head()

,four_people,median_income,one_person,percent_college,percent_lunch,percent_married,rate_unemployment,state,three_people,two_people,year,zip_code,average_act
2495,915.0,52833.0,875.0,0.564717,0.226481,0.823245,0.100518,MI,602.0,1431.0,2016-2017,49112,21.000000
3715,4526.0,62411.0,1975.0,0.537197,0.677895,0.313253,0.095582,NY,2405.0,3227.0,2016-2017,11413,16.245421
4860,52.0,63938.0,1394.0,0.781818,0.561431,0.523810,0.096433,PA,28.0,890.0,2016-2017,15222,18.345543
3722,2299.0,25625.0,5240.0,0.361014,0.625239,0.317358,0.168471,NY,2467.0,6107.0,2016-2017,11103,18.663004
1814,1545.0,46350.0,2182.0,0.602669,0.358377,0.641444,0.089737,KY,1211.0,3222.0,2016-2017,42025,20.000000


##Export the clean .csv file

Creating the csv files for df_train & df_test and saving it

In [ ]:
from google.colab import files

df_train.to_csv('clean_train.csv', encoding = 'utf-8-sig', index=False) 
files.download('clean_train.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_test.to_csv('clean_test.csv', encoding = 'utf-8-sig', index=False) 
files.download('clean_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>